
docker cp  /home/yury/Downloads/FIT5202-Project/big_data_tutorials/normalise.ipynb bd_cont:/home/normalise.ipynb

docker cp  /home/yury/Downloads/FIT5202-Project/big_data_tutorials/part-00000-dfc11f65-5287-4c73-be9f-d12bac5d8fff-c000.csv bd_cont:/home/part-00000-dfc11f65-5287-4c73-be9f-d12bac5d8fff-c000.csv

In [1]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types.DoubleType

Intitializing Scala interpreter ...

Spark Web UI available at http://dd15ded6b68d:4042
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591097372530)
SparkSession available as 'spark'


import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types.DoubleType


In [2]:
val weather_org_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("part-00000-dfc11f65-5287-4c73-be9f-d12bac5d8fff-c000.csv")

weather_org_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


In [3]:
weather_org_df.registerTempTable("Aus_Rain_Data")

<font color='red'>
YURY =====================================      MY CODE STARTS     =====================================
 </font>

In [4]:
val df0 = spark.sql("select * from Aus_Rain_Data")

df0: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


In [5]:
val df1 = df0.withColumn("Pressure9amTmp", df0("Pressure9am").cast(DoubleType))
    .drop("Pressure9am")
    .withColumnRenamed("Pressure9amTmp", "Pressure9am")

val df2 = df1.withColumn("Pressure3pmTmp", df1("Pressure3pm").cast(DoubleType))
    .drop("Pressure3pm")
    .withColumnRenamed("Pressure3pmTmp", "Pressure3pm")

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]
df2: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


In [6]:
val min_P9am = df2.agg(min("Pressure9am") as "min_P9am").select(col("min_P9am")).first.getDouble(0)
val max_P9am = df2.agg(max("Pressure9am") as "max_P9am").select(col("max_P9am")).first.getDouble(0)

2020-06-02 11:29:49,687 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


min_P9am: Double = 980.5
max_P9am: Double = 1041.0


In [7]:
val min_P3pm = df2.agg(min("Pressure3pm") as "min_P3pm").select(col("min_P3pm")).first.getDouble(0)
val max_P3pm = df2.agg(max("Pressure3pm") as "max_P3pm").select(col("max_P3pm")).first.getDouble(0)

min_P3pm: Double = 977.1
max_P3pm: Double = 1039.6


In [8]:
val minPRange = 0    //from the report  /* 0 970.0
val maxPRange = 1   //from the report  /* 1 970.0

minPRange: Int = 0
maxPRange: Int = 1


\begin{equation}
  Rescaled_{9am} = \frac{Pressure_i - Pressure_{min9am}}{Pressure_{max9am} - Pressure_{min9am}} * (maxPressureRange - minPressureRange) + minPressureRange
\end{equation}

In [9]:
val df3 = df2.withColumn("Pressure3pm_Sc", 
    (((df2("Pressure3pm") - min_P3pm) / (max_P3pm -min_P3pm)) * (maxPRange - minPRange)) + minPRange)
/*    .drop("Pressure3pm")
    .withColumnRenamed("Pressure3pmTmp", "Pressure3pm") */

df3: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 24 more fields]


In [10]:
val df4 = df3.withColumn("Pressure9am_Sc", 
    (((df2("Pressure9am") - min_P9am) / (max_P9am -min_P9am)) * (maxPRange - minPRange)) + minPRange)

/*.drop("Pressure9am")
    .withColumnRenamed("Pressure9amTmp", "Pressure9am") */

df4: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 25 more fields]


In [11]:
df4.coalesce(1)
      .write
      .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("normalised")

In [12]:
!ls /home/normalised

_SUCCESS  part-00000-783a7de0-de76-471b-ae96-4e1854e7ac04-c000.csv



In [13]:
!mv /home/normalised/*.csv /home/normalised/weatherAUS_N.csv

In [14]:
!mv /home/normalised/weatherAUS_N.csv /home/weatherAUS_N.csv

In [15]:
!wc -l   weatherAUS_N.csv
!wc -l   weatherAUS.csv

142194 weatherAUS_N.csv

wc: weatherAUS.csv: No such file or directory



In [16]:
!rm -r /home/normalised/

<font color='red'>
YURY=====================================      MY CODE ENDS     =====================================
 </font>


docker cp bd_cont:/home/normalise.ipynb /home/yury/Downloads/FIT5202-Project/big_data_tutorials/normalise.ipynb

docker cp bd_cont:/home/weatherAUS_N.csv /home/yury/Downloads/FIT5202-Project/big_data_tutorials/weatherAUS_N.csv

